In [22]:
from __future__ import print_function
import cv2
import numpy as np
from skimage.metrics import structural_similarity
import matplotlib.pyplot as plt
from cv2 import cv2 as cv2
from PIL import Image
import imagehash
from moviepy.editor import VideoFileClip
import os
from collections import deque

## parameter

In [23]:
gap_factor=10 ## num of detection within each second
match_ratio=0.3 ##the distance ratio thershold between best match and second match
good_match_rate=0.7 ##the rate of matches to be chosen in all
ransacReprojThreshold=5
min_match_count=5


## anotation

for keypoint, attribute to use are:
    pt 
    angle
    response
    size
    octave
    class_id：object
for match, attribute to use are:
    distance
    queryIdx
    trainIdx
    imgIdx
    
    print(kp1[i.queryIdx].pt) location before move
    print(kp2[i.trainIdx].pt) location after move

## test begin

In [88]:
def main():
    
    ##load test video
    video=os.path.abspath(os.curdir)+'/click_in.mp4'
    videoCapture = cv2.VideoCapture(video)
    
    ##detect for gap_factor #gap_dected frames
    fps = videoCapture.get(cv2.CAP_PROP_FPS)
    print(fps)
    gap_dected=int(fps//gap_factor)
    
    ##load first frame
    success, frame_old = videoCapture.read()
    i,j=0,0
    _SaveImage(frame_old,os.path.abspath(os.curdir)+'/test',j)
    print('save image:',i)
    
    predict=deque([],maxlen=gap_factor)
    
    while success:
        i+=1
    ##finish process if all frames loaded
        success, frame_new = videoCapture.read()
        if success==False:
            print('vedio detection finish')
#           sys.exit() 
            return 
        if (i % gap_dected == 0):
            j+=1
            _SaveImage(frame_new,os.path.abspath(os.curdir)+'/test',j)
            print('save image:',i)
            
    ##predict 
            if _AlignImages(frame_old,frame_new) != None:
                predict.append(_AlignImages(frame_old,frame_new))
                if len(predict)==gap_factor:
                    res=_Predict(predict)
                    print(i//30,'s',i%30,'frame',res)
            
            
        frame_old=frame_new
    
            
        
        

        
        
    
    

In [89]:
def _SaveImage(image,addr,num):
    address = addr +'/'+ str(num)+ '.jpg'
    cv2.imwrite(address,image)

def _AlignImages(img1,img2):
    # initiate ORB detector
    orb = cv2.ORB_create(5000)
    
    ##check bilateral banner, eliminate stick part
#     up_idx,down_idx=_SplitImage(img1,img2)
#     img1 = img1[up_idx:down_idx,:]
#     img2= img2[up_dx:down_idx,:]
    img1 = img1[80:440,:]
    img2 = img2[80:440,:]
    
    ##find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)
    if des2 is None: 
        return 'click','in'
    
    ##match with Brute-Force
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    ##find best 2 matches for each kp
    matches = bf.knnMatch(des1, trainDescriptors = des2, k = 2)
    print(len(matches))
    ##ratio test
    good = [m for (m,n) in matches if m.distance < match_ratio*n.distance]
    ##sort and choose the best part of matches
#     good = sorted(good, key = lambda x:x.distance)
#     good = good[:int(len(good) * good_match_rate)]
    
    if len(good)>min_match_count:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        ##find (3x3) transform matrix
        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,ransacReprojThreshold)
        t_x = M[0][2]
        t_y = M[1][2]
        return (t_x,t_y)
    else:
        return None
        
        

def _Predict(predict):
    res=[]
    for p in predict:
        res.append(__Pattern(p[0],p[1]))
    return res
    
def __Pattern(t_x,t_y):
    if type(t_x)!=str and type(t_y)!=str:
        if abs(t_x)<1 and abs(t_y)<1:
    #         print('stop')
            return 'stop'
        elif abs(t_x/t_y)>15:
            if (t_x>0):
    #             print('down')
                return 'right'
            else:
    #             print('up')
                return 'left'

        elif abs(t_y/t_x)>15:
    #         print('up,down')
            if (t_y>0):
    #             print('down')
                return 'down'
            else:
    #             print('up')
                return 'up'
        else:
            return 'stop'
    else:
        if t_x=='click' and t_y=='in':
            return t_x+t_y
        
        
          
    
    
    
    
    
    
    
    

In [90]:
main()

26.25
save image: 0
save image: 2
3863
3863
save image: 4
3862
3862
save image: 6
3866
3866
save image: 8
3884
3884
save image: 10
3887
3887
save image: 12
3883
3883
save image: 14
3874
3874
save image: 16
3885
3885
save image: 18
3882
3882
save image: 20
3869
3869
0 s 20 frame ['stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop']
save image: 22
3874
3874
0 s 22 frame ['stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop']
save image: 24
3872
3872
0 s 24 frame ['stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop']
save image: 26
3870
3870
0 s 26 frame ['stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop']
save image: 28
3870
3870
0 s 28 frame ['stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'left']
save image: 30
3852
3852
1 s 0 frame ['stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'left', 'left']
save image: 32
3509
3509
1 s 2 frame ['stop', 

In [73]:
a=deque([],maxlen=2)


In [76]:
a.append(1)
a.append(None)

In [77]:
a


deque([1, None])

In [20]:
test()

'stop'

In [82]:
a=1

In [83]:
type(a)

int

In [84]:
type(str(a))

str

In [85]:
type(a)!=str

True